This project is written by Rui Bai(rb3454) and Yichi Liu (yl4327). The logic is based on the [paper](https://arxiv.org/abs/1703.02442). We used data from [folder](https://drive.google.com/drive/folders/1rwWL8zU9v0M27BtQKI52bF6bVLW82RL5?usp=sharing) which contains several slides from [CAMELYON16](https://camelyon17.grand-challenge.org/Data/) dataset.

After checking the slides manually, we found that most slides do not contain cancer cell. we chose 8 slides for training and  validation and 3 slides for testing.

For each slides, we randomly selected 200 positive patches and 200 negative patches for further model constructions.

This script will extract patches from 2 selected zoom levels. 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Install the OpenSlide C library and Python bindings
# After installing these libraries, use `Runtime -> restart and run all` on the menu
!apt-get install openslide-tools
!pip install openslide-python

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from openslide import open_slide, __library_version__ as openslide_version
import os
from PIL import Image
from skimage.color import rgb2gray

import random
from tqdm.notebook import tqdm

import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

BATCH_SIZE = 32
PATCH_SIZE = 299
CENTER_SIZE = 128

## Patches Extraction

In [ ]:
""""
Cited from Starter Code
"""
# See https://openslide.org/api/python/#openslide.OpenSlide.read_region
# Note: x,y coords are with respect to level 0.
# There is an example below of working with coordinates
# with respect to a higher zoom level.

# Read a region from the slide
# Return a numpy RBG array
def read_slide(slide, x, y, level, width, height, as_float=False):
    im = slide.read_region((x,y), level, (width, height))
    im = im.convert('RGB') # drop the alpha channel
    if as_float:
        im = np.asarray(im, dtype=np.float32)
    else:
        im = np.asarray(im)
    assert im.shape == (height, width, 3)
    return im

In [ ]:
""""
Cited from Starter Code
"""

# As mentioned in class, we can improve efficiency by ignoring non-tissue areas 
# of the slide. We'll find these by looking for all gray regions.
def find_tissue_pixels(image, intensity=0.8):
    im_gray = rgb2gray(image)
    assert im_gray.shape == (image.shape[0], image.shape[1])
    indices = np.where(im_gray <= intensity)
    return list(zip(indices[0], indices[1])) 

def apply_mask(im, mask, color=(255,0,0)):
    masked = np.copy(im)
    for x,y in mask: masked[x][y] = color
    return masked


In [ ]:
def get_a_patch_from_a_slide(slide, tumor_mask, x, y, size = 299, level = 4):
  # (x,y) is the upper-left corner of the desired patch
  # return the patch for slide, tumor mask, and tissue
  
  patch_slide = read_slide(slide, x=x, y=y, level=level, width=size, height=size)
  patch_mask = read_slide(tumor_mask, x=x, y=y, level=level, width=size, height=size)[:,:,0]

  return patch_slide, patch_mask

In [ ]:
# This function could aolve the memory problem
def get_multiscale_pos_neg_patches_from_a_slide(slide, tumor_mask, size = PATCH_SIZE, center_size = CENTER_SIZE, levels = [2, 3], num_pos = 100, num_neg = 100):
  '''
  Returns dictionaries of lists: key is scale level, value is the list of patches for thats scale level.
  The criteron to choose the patches is based on the 1st scale level in input 'levels'.
  '''
  # using 1st scale level as reference, choose the center coordinates from the first elements in levels
  ref_level = levels[0]
  ref_width, ref_height = slide.level_dimensions[ref_level]
  half_patch_size = size // 2
  half_center_size = center_size // 2

  ref_scale_factor = 2**ref_level

  # create dictionary to store patch images
  pos_patches = {level:[] for level in levels}
  neg_patches = {level:[] for level in levels}
  
  # create lists to store patch center coordinates
  neg_center_coord = []
  pos_center_coord = []

  ### to get pos patches ###
  print("getting pos patches...")

  # get random pos pixels to be the center of desired pos patches
  random.seed(1217)
  random_pos_pixels = []
  while len(random_pos_pixels) < num_pos:
    # random a pixel if that point is tumor, add to list
    random_pos_pixel = (random.sample(range(ref_width),1)[0], random.sample(range(ref_height),1)[0])
    mask_pixel = read_slide(tumor_mask, x=random_pos_pixel[0]*ref_scale_factor, y=random_pos_pixel[1]*ref_scale_factor,
                            level=ref_level, width=1, height=1)[0,0,0]
    if mask_pixel == 1:
      random_pos_pixels.append(random_pos_pixel)

  # for each random pos pixel, get the patch that is centered at that pixel
  for pixel in tqdm(random_pos_pixels):
    # calculate the level 0 upper-left corner coordinate using this pixel as center coordinate (this pixel coordinate is w.r.t current level)
    center_level_0_coord = (pixel[0] * ref_scale_factor, pixel[1] * ref_scale_factor)
    pos_center_coord.append(center_level_0_coord)
    for level in levels:
        # get the patch for each level
        scale_factor = 2 ** level
        x_upper_left = center_level_0_coord[0] - half_patch_size * scale_factor
        y_upper_left = center_level_0_coord[1] - half_patch_size * scale_factor
        patch_slide, _ = get_a_patch_from_a_slide(slide, tumor_mask, x_upper_left, y_upper_left, size, level)
        pos_patches[level].append(patch_slide)
    
  
  ### to get neg patches ###
  print("getting neg patches...")

  # get random neg pixels to be the center of desired neg patches, only using tissue pixels
  
  # due to the ram, detect whether it is a tissue area, not done!!!
  random.seed(1217)
  
  random_neg_pixels = []
  while len(random_neg_pixels) < num_neg:
    # random a pixel
    random_neg_pixel = (random.sample(range(ref_width),1)[0], random.sample(range(ref_height),1)[0])
    image_pixel = read_slide(slide, x=random_neg_pixel[0]*ref_scale_factor, y=random_neg_pixel[1]*ref_scale_factor,
                            level=ref_level, width=1, height=1)
    # if the intensity of this pixels <= 0.8, i.e. it is a tissue, take this into consideration
    im_pixel_gray = rgb2gray(image_pixel)
    if im_pixel_gray[0][0] <= 0.8:
      # get the mask of the center part (128*128) using this pixel as center
      x_upper_left = random_neg_pixel[0] * ref_scale_factor - half_center_size * ref_scale_factor
      y_upper_left = random_neg_pixel[1] * ref_scale_factor - half_center_size * ref_scale_factor
      _, patch_mask = get_a_patch_from_a_slide(slide, tumor_mask, x_upper_left, y_upper_left, center_size, ref_level)
      # if the center area do not contains tumor, add to list
      if 1 not in patch_mask:
        random_neg_pixels.append(random_neg_pixel)

  # for each random pos pixel, get the patch that is centered at that pixel
  for pixel in tqdm(random_neg_pixels):
    # calculate the level 0 upper-left corner coordinate using this pixel as center coordinate (this pixel coordinate is w.r.t current level)
    center_level_0_coord = (pixel[0] * ref_scale_factor, pixel[1] * ref_scale_factor)
    neg_center_coord.append(center_level_0_coord)
    for level in levels:
      scale_factor = 2 ** level
      x_upper_left = center_level_0_coord[0] - half_patch_size * scale_factor
      y_upper_left = center_level_0_coord[1]- half_patch_size * scale_factor
      patch_slide, _ = get_a_patch_from_a_slide(slide, tumor_mask, x_upper_left, y_upper_left, size, level)
      neg_patches[level].append(patch_slide)
  print("finished.\n\n")

  return pos_patches, pos_center_coord, neg_patches, neg_center_coord

In [ ]:
def get_patches_from_multiple_slides_save(slide_index_list, levels, train_dir_l1, train_dir_l2, val_dir_l1, val_dir_l2,
                                          num_pos_per_slide = 100, num_neg_per_slide = 100):
  l1 = levels[0]
  l2 = levels[1]


  if not os.path.exists(train_dir_l1):
    os.makedirs(train_dir_l1)
    os.makedirs(train_dir_l1 + '/1')
    os.makedirs(train_dir_l1 + '/0')
  if not os.path.exists(train_dir_l2):
    os.makedirs(train_dir_l2)
    os.makedirs(train_dir_l2 + '/1')
    os.makedirs(train_dir_l2 + '/0')
  if not os.path.exists(val_dir_l1):
    os.makedirs(val_dir_l1)
    os.makedirs(val_dir_l1 + '/1')
    os.makedirs(val_dir_l1 + '/0')
  if not os.path.exists(val_dir_l2):
    os.makedirs(val_dir_l2)
    os.makedirs(val_dir_l2 + '/1')
    os.makedirs(val_dir_l2 + '/0')
  for slide_num in slide_index_list:
    print("Processing slide:", slide_num)
    # open the slide from drive
    slide_path = '/content/gdrive/My Drive/applied deep learning/final pj/data/tumor_' + slide_num + '.tif'
    tumor_mask_path = '/content/gdrive/My Drive/applied deep learning/final pj/data/tumor_' + slide_num + '_mask.tif'
    slide = open_slide(slide_path)
    tumor_mask = open_slide(tumor_mask_path)
    print("Done opening slides")
    # extract multi-scale pos and neg patches from this slide
    pos_patches, pos_center_coord, neg_patches, neg_center_coord = get_multiscale_pos_neg_patches_from_a_slide(slide, tumor_mask, size = PATCH_SIZE, 
                                                                                                               center_size = 128, levels = levels, 
                                                                                                               num_pos = num_pos_per_slide, 
                                                                                                              num_neg = num_neg_per_slide)
    print("Done getting coords")
    train_idxs_pos, val_idxs_pos = train_test_split(range(len(pos_center_coord)),test_size=0.2, shuffle=True)
    train_idxs_neg, val_idxs_neg = train_test_split(range(len(neg_center_coord)),test_size=0.2, shuffle=True)


    # save the patches
    for i in train_idxs_pos:
      center = pos_center_coord[i]
      Image.fromarray(pos_patches[l1][i]).save(train_dir_l1 +'/1/slide_' + str(slide_num) +
                                               '_level' + str(l1) + 
                                               '_' + str(center[0]) +
                                               '_' + str(center[1])+ '.jpeg')
      Image.fromarray(pos_patches[l2][i]).save(train_dir_l2 +'/1/slide_' + str(slide_num) +
                                               '_level' + str(l2) + 
                                               '_' + str(center[0]) +
                                               '_' + str(center[1])+ '.jpeg')
    for i in train_idxs_neg:
      center = pos_center_coord[i]
      Image.fromarray(neg_patches[l1][i]).save(train_dir_l1 + '/0/slide_' + str(slide_num) +
                                               '_level' + str(l1) + 
                                               '_' + str(center[0]) +
                                               '_' + str(center[1])+ '.jpeg')
      Image.fromarray(neg_patches[l2][i]).save(train_dir_l2 + '/0/slide_' + str(slide_num) +
                                               '_level' + str(l2) + 
                                               '_' + str(center[0]) +
                                               '_' + str(center[1])+ '.jpeg')
    for i in val_idxs_pos:
      center = pos_center_coord[i]
      Image.fromarray(pos_patches[l1][i]).save(val_dir_l1 + '/1/slide_' + str(slide_num) +
                                               '_level' + str(l1) + 
                                               '_' + str(center[0]) +
                                               '_' + str(center[1])+ '.jpeg')
      Image.fromarray(pos_patches[l2][i]).save(val_dir_l2 + '/1/slide_'+ str(slide_num) +
                                               '_level' + str(l2) + 
                                               '_' + str(center[0]) +
                                               '_' + str(center[1])+ '.jpeg')
    for i in val_idxs_neg:
      center = pos_center_coord[i]
      Image.fromarray(neg_patches[l1][i]).save(val_dir_l1 + '/0/slide_' + str(slide_num) +
                                               '_level' + str(l1) + 
                                               '_' + str(center[0]) +
                                               '_' + str(center[1])+ '.jpeg')
      Image.fromarray(neg_patches[l2][i]).save(val_dir_l2 + '/0/slide_' + str(slide_num) +
                                               '_level' + str(l2) + 
                                               '_' + str(center[0]) +
                                               '_' + str(center[1])+ '.jpeg')
  #   all_pos_patches[l1] += pos_patches[l1]
  #   all_pos_patches[l2] += pos_patches[l2]
  #   all_neg_patches[l1] += neg_patches[l1]
  #   all_neg_patches[l2] += neg_patches[l2]

  #   all_pos_center_coord += pos_center_coord
  #   all_neg_center_coord += neg_center_coord

  # # plot the 1st pos patch with 1st level
  # plt.imshow(all_pos_patches[l1][0])

  # return all_pos_center_coord, all_neg_patches, all_neg_center_coord

In [ ]:
multi_levels = [1,2]

levels = 'level12'
level1 = 'level1'
level2 = 'level2'


train_dir_l1 = '/content/gdrive/My Drive/applied deep learning/final pj/'+levels + '/' + level1+'/train'
train_dir_l2 = '/content/gdrive/My Drive/applied deep learning/final pj/'+levels + '/' + level2+'/train'
val_dir_l1 = '/content/gdrive/My Drive/applied deep learning/final pj/'+levels + '/' + level1+'/validation'
val_dir_l2 = '/content/gdrive/My Drive/applied deep learning/final pj/'+levels + '/' + level2+'/validation'

###uncomment below to save patches to drive!!

In [ ]:
# the shape of each patch is (299,299,3)

l1 = multi_levels[0]
l2 = multi_levels[1]

# Get multi-scale patches from a list of slide images
# test_indices = ['091', '096','110']

indices = [ '016','031', '064', '075', '078', '084', '094', '101']
get_patches_from_multiple_slides_save(indices, multi_levels, train_dir_l1, train_dir_l2, val_dir_l1, val_dir_l2, 200, 200)

Processing slide: 016
Done opening slides
getting pos patches...



getting neg patches...



finished.


Done getting coords
Processing slide: 031
Done opening slides
getting pos patches...



getting neg patches...



finished.


Done getting coords
Processing slide: 064
Done opening slides
getting pos patches...


...


finished.


Done getting coords
Processing slide: 094
Done opening slides
getting pos patches...



getting neg patches...



finished.


Done getting coords
Processing slide: 101
Done opening slides
getting pos patches...



getting neg patches...



finished.


Done getting coords
